In [485]:
import cvxpy as cp
import numpy as np
from numpy.typing import NDArray
import random

def get_ordered_comparison_model(line_array : NDArray, epsilon=1e-3, M = 1e8):
    ordered_list = get_list_of_arrays_less_than_k(line_array)
    Cis = cp.Variable((line_array.shape[0],len(ordered_list)), boolean=True, name=f'c{random.randrange(1,1000)}')
    tower_visibility = cp.Variable(line_array.shape[0], boolean=True)
    constraints_hi = []
    for idx in range(len(ordered_list)):
        # if and only if
        constraints_hi += [ordered_list[idx][idx] - ordered_list[idx][k] + epsilon<= M*(Cis[idx,k])
                          for k in range(ordered_list[idx].shape[0])]
        constraints_hi += [ordered_list[idx][idx] - ordered_list[idx][k] + epsilon >= -M*(1-Cis[idx,k])
                          for k in range(ordered_list[idx].shape[0])]
        # fill the rest of the triangle with empty
        constraints_hi += [Cis[idx, k] == 0 for k in range(ordered_list[idx].shape[0], len(ordered_list))]

        constraints_hi += [cp.sum(Cis[idx, :]) - (idx+1) + epsilon <= M*(tower_visibility[idx])]
        constraints_hi += [cp.sum(Cis[idx, :]) - (idx+1) + epsilon >= -M*(1-tower_visibility[idx])]
       
    return tower_visibility, Cis, constraints_hi

def get_list_of_arrays_less_than_k(line_array : NDArray):
    return [line_array[:k+1] for k in range(line_array.shape[0])]

# Example data
A = np.array([1, 4, 3, 2]) 
print(get_list_of_arrays_less_than_k(A))
epsilon = 1e-5  # Tolerance for strict inequality
M = 1e5  # Big-M value

tower_vis, Cis, constraints = get_ordered_comparison_model(A, epsilon, M)
objective = cp.Maximize(cp.sum(tower_vis))
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCIP)

print(problem.status)

for c in problem.constraints:
    print(str(c))
    
for vars in problem.variables():
    print(f"{vars} : {vars.value}")

# Results
print(f"Number of towers to see: {problem.value}")

[array([1]), array([1, 4]), array([1, 4, 3]), array([1, 4, 3, 2])]
optimal
1e-05 <= 100000.0 @ c120[0, 0]
-100000.0 @ (1.0 + -c120[0, 0]) <= 1e-05
c120[0, 1] == 0.0
c120[0, 2] == 0.0
c120[0, 3] == 0.0
Sum(c120[0, 0:4], None, False) + -1.0 + 1e-05 <= 100000.0 @ var970259[0]
-100000.0 @ (1.0 + -var970259[0]) <= Sum(c120[0, 0:4], None, False) + -1.0 + 1e-05
3.00001 <= 100000.0 @ c120[1, 0]
1e-05 <= 100000.0 @ c120[1, 1]
-100000.0 @ (1.0 + -c120[1, 0]) <= 3.00001
-100000.0 @ (1.0 + -c120[1, 1]) <= 1e-05
c120[1, 2] == 0.0
c120[1, 3] == 0.0
Sum(c120[1, 0:4], None, False) + -2.0 + 1e-05 <= 100000.0 @ var970259[1]
-100000.0 @ (1.0 + -var970259[1]) <= Sum(c120[1, 0:4], None, False) + -2.0 + 1e-05
2.00001 <= 100000.0 @ c120[2, 0]
-0.99999 <= 100000.0 @ c120[2, 1]
1e-05 <= 100000.0 @ c120[2, 2]
-100000.0 @ (1.0 + -c120[2, 0]) <= 2.00001
-100000.0 @ (1.0 + -c120[2, 1]) <= -0.99999
-100000.0 @ (1.0 + -c120[2, 2]) <= 1e-05
c120[2, 3] == 0.0
Sum(c120[2, 0:4], None, False) + -3.0 + 1e-05 <= 100000.0 @

In [486]:
import cvxpy as cp

def make_discrete_decision_variables(array_shape, allowed_values):
    # Binary variables: z[i,j,k] = 1 if C[i,j] == allowed_values[k]
    z = cp.Variable( array_shape + tuple([len(allowed_values)]), boolean=True, name='z')
    constraints = []
    for i in range(array_shape[0]):
        for j in range(array_shape[1]):
            # Only one value can be chosen per C[i,j]
            constraints += [cp.sum(z[i, j, :]) == 1]
    return z, constraints

def guarantee_one_of_each_in_full_matrix(z_array, allowed_values):
    constraints = []
    # for each column (it is supposed to be switched like that)
    for row in range(z_array.shape[0]):
        # Each value v must appear at least once in C
        for v_idx, _ in enumerate(allowed_values):
            constraints += [cp.sum(z_array[row,:, v_idx]) == 1]  # one occurrence of v
    
    # for each row (it is supposed to be switched like that)
    for column in range(z_array.shape[0]):
        # Each value v must appear at least once in C
        for v_idx, _ in enumerate(allowed_values):
            constraints += [cp.sum(z_array[:,column, v_idx]) == 1]  # one occurrence of v
    return constraints

def get_final_solution_constraints(decision_variables):
    final_result = cp.Variable(decision_variables[:,:,0].shape, integer=True, name='final')
    final_result_arr = np.zeros(decision_variables[:,:,0].shape)

    for idx in range(decision_variables.shape[2]):
        final_result_arr = final_result_arr + decision_variables[:,:, idx]*allowed_values[idx]

    constraint = [final_result == final_result_arr]
    
    return final_result, constraint

n = 4
allowed_values = [1, 2, 3, 4]  # Discrete set
decision_variables, constraints = make_discrete_decision_variables((n, n), allowed_values)
constraints += guarantee_one_of_each_in_full_matrix(decision_variables, allowed_values)

final_solution, final_solution_constraints = get_final_solution_constraints(decision_variables)
constraints += final_solution_constraints

z = decision_variables

# fill some just to test
test_contraints = [z[1, 1, 2] == 1, z[1, 2, 1] == 1, z[1, 3, 0] == 1, z[2, 3, 2] == 1,
                   z[3, 1, 3] == 1, z[0, 1, 0] == 1]
constraints += test_contraints

objective = cp.Maximize(cp.sum(decision_variables))

problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCIP)

print(problem.status)
print(final_solution.value)


optimal
[[2. 1. 3. 4.]
 [4. 3. 2. 1.]
 [1. 2. 4. 3.]
 [3. 4. 1. 2.]]


/home/pedro.trindade/anaconda3/envs/optmization/lib/python3.13/site-packages/cvxpy/reductions/solvers/solving_chain.py:418: UserWarning: The problem has an expression with dimension greater than 2. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


In [494]:
n = 5
constraints = []

side_towers = np.zeros((n, 2, 2)) #position, row or column, orientation (right or left, down or up)

def get_side_towers_constraints(final_result_vars, side_towers):
    constraints = []
    total_sum = cp.Variable(side_towers.shape, integer=True, name='tower_sums')
    for position in range(side_towers.shape[0]):
        for pos_type in range(side_towers.shape[1]):
            for orientation in range(side_towers.shape[2]):
                towers_value = side_towers[position, pos_type, orientation]
                if not(towers_value == 0):
                    if pos_type == 0:
                        if orientation == 0:
                            tower_vis, _, tower_constraint = get_ordered_comparison_model(final_result_vars[position, :])
                        else:
                            tower_vis, _, tower_constraint = get_ordered_comparison_model(final_result_vars[position,::-1])
                    else:
                        if orientation == 0:
                            tower_vis, _, tower_constraint = get_ordered_comparison_model(final_result_vars[:, position])
                        else:
                            tower_vis, _, tower_constraint = get_ordered_comparison_model(final_result_vars[::-1, position])

                    constraints += [total_sum[position, pos_type, orientation] <= cp.sum(tower_vis)]
                    constraints += tower_constraint
                    constraints += [towers_value == cp.sum(tower_vis)]
                else:
                    constraints += [total_sum[position, pos_type, orientation] == 0]

    return total_sum, constraints

## Test
s = side_towers
## Linhas
#s[1,0,0] = 2
#s[3,0,1] = 4
s[1,1,0] = 2
s[3,1,0] = 2
s[2,0, 1] = 3
s[3,0,1] = 1
s[3,0,0] = 2

## Colunas 
#s[1,1,0] = 1
#s[0,1,0] = 3
#s[3,1,1] = 2

side_towers = s

n = 4
allowed_values = [1, 2, 3, 4]  # Discrete set
decision_variables, constraints = make_discrete_decision_variables((n, n), allowed_values)
constraints += guarantee_one_of_each_in_full_matrix(decision_variables, allowed_values)

final_solution, final_solution_constraints = get_final_solution_constraints(decision_variables)
constraints += final_solution_constraints

total_sum, tower_constraints = get_side_towers_constraints(final_solution, side_towers)
constraints += tower_constraints

for c in tower_constraints:
    print(str(c))

objective = cp.Maximize(cp.sum(decision_variables) + cp.sum(total_sum))

problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCIP)

print(problem.status)
for vars in problem.variables()[2:20]:
    print(f"{vars} : {vars.value}")
print(final_solution.value)


tower_sums[0, 0, 0] == 0.0
tower_sums[0, 0, 1] == 0.0
tower_sums[0, 1, 0] == 0.0
tower_sums[0, 1, 1] == 0.0
tower_sums[1, 0, 0] == 0.0
tower_sums[1, 0, 1] == 0.0
tower_sums[1, 1, 0] <= Sum(var1002588, None, False)
final[0:4, 1][0][0] + -final[0:4, 1][0][0] + 0.001 <= 100000000.0 @ c49[0, 0]
-100000000.0 @ (1.0 + -c49[0, 0]) <= final[0:4, 1][0][0] + -final[0:4, 1][0][0] + 0.001
c49[0, 1] == 0.0
c49[0, 2] == 0.0
c49[0, 3] == 0.0
Sum(c49[0, 0:4], None, False) + -1.0 + 0.001 <= 100000000.0 @ var1002588[0]
-100000000.0 @ (1.0 + -var1002588[0]) <= Sum(c49[0, 0:4], None, False) + -1.0 + 0.001
final[0:4, 1][0:2][1] + -final[0:4, 1][0:2][0] + 0.001 <= 100000000.0 @ c49[1, 0]
final[0:4, 1][0:2][1] + -final[0:4, 1][0:2][1] + 0.001 <= 100000000.0 @ c49[1, 1]
-100000000.0 @ (1.0 + -c49[1, 0]) <= final[0:4, 1][0:2][1] + -final[0:4, 1][0:2][0] + 0.001
-100000000.0 @ (1.0 + -c49[1, 1]) <= final[0:4, 1][0:2][1] + -final[0:4, 1][0:2][1] + 0.001
c49[1, 2] == 0.0
c49[1, 3] == 0.0
Sum(c49[1, 0:4], None, Fa

/home/pedro.trindade/anaconda3/envs/optmization/lib/python3.13/site-packages/cvxpy/reductions/solvers/solving_chain.py:418: UserWarning: The problem has an expression with dimension greater than 2. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


optimal
final : [[1. 2. 4. 3.]
 [2. 4. 3. 1.]
 [4. 3. 1. 2.]
 [3. 1. 2. 4.]]
var1002588 : [ 1.  1. -0. -0.]
c49 : [[ 1. -0. -0. -0.]
 [ 1.  1. -0. -0.]
 [ 1. -0.  1. -0.]
 [-0. -0. -0.  1.]]
var1002982 : [ 1. -0.  1.  1.]
c436 : [[ 1. -0. -0. -0.]
 [ 0.  1. -0. -0.]
 [ 1.  1.  1. -0.]
 [ 1.  1.  1.  1.]]
var1003376 : [ 1.  0. -0.  1.]
c275 : [[ 1. -0. -0. -0.]
 [-0.  1. -0. -0.]
 [-0.  1.  1. -0.]
 [ 1.  1.  1.  1.]]
var1003760 : [ 1. -0. -0. -0.]
c609 : [[ 1. -0. -0. -0.]
 [-0.  1. -0. -0.]
 [-0.  0.  1. -0.]
 [-0.  1.  1.  1.]]
var1004144 : [ 1. -0. -0.  1.]
c219 : [[ 1. -0. -0. -0.]
 [ 0.  1. -0. -0.]
 [-0.  1.  1. -0.]
 [ 1.  1.  1.  1.]]
[[1. 2. 4. 3.]
 [2. 4. 3. 1.]
 [4. 3. 1. 2.]
 [3. 1. 2. 4.]]
